# 자체 평가지표
* 가지고 있는 제출 파일과 비교

## 데이터 가져오기

In [40]:
import pandas as pd
from tqdm import tqdm

In [41]:
before_sub = pd.read_csv('../07_결과_출력/sub23_pycaret11.csv')
present_sub = pd.read_csv('../07_결과_출력/sub24_pycaret12.csv')

In [42]:
before_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   일자|시간|구분  15120 non-null  object 
 1   공급량       15120 non-null  float64
dtypes: float64(1), object(1)
memory usage: 236.4+ KB


In [43]:
present_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   일자|시간|구분  15120 non-null  object 
 1   공급량       15120 non-null  float64
dtypes: float64(1), object(1)
memory usage: 236.4+ KB


In [44]:
before_sub.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,1986.488313
1,2019-01-01 02 A,1777.625538
2,2019-01-01 03 A,1710.387693
3,2019-01-01 04 A,1777.399099
4,2019-01-01 05 A,1936.957736


## 열을 일자, 시간, 구분 열 분해

In [45]:
# 열 정리
before_sub[['일자', '시간', '구분']] = before_sub['일자|시간|구분'].str.split(' ').to_list()
before_sub.head()

,일자|시간|구분,공급량,일자,시간,구분
0,2019-01-01 01 A,1986.488313,2019-01-01,01,A
1,2019-01-01 02 A,1777.625538,2019-01-01,02,A
2,2019-01-01 03 A,1710.387693,2019-01-01,03,A
3,2019-01-01 04 A,1777.399099,2019-01-01,04,A
4,2019-01-01 05 A,1936.957736,2019-01-01,05,A


In [46]:
before_sub = before_sub.drop('일자|시간|구분', axis=1)
before_sub = before_sub[['일자', '시간', '구분', '공급량']]
before_sub.head()

,일자,시간,구분,공급량
0,2019-01-01,01,A,1986.488313
1,2019-01-01,02,A,1777.625538
2,2019-01-01,03,A,1710.387693
3,2019-01-01,04,A,1777.399099
4,2019-01-01,05,A,1936.957736


In [47]:
# 열 정리
present_sub[['일자', '시간', '구분']] = present_sub['일자|시간|구분'].str.split(' ').to_list()
present_sub = present_sub.drop('일자|시간|구분', axis=1)
present_sub = present_sub[['일자', '시간', '구분', '공급량']]
present_sub.head()

,일자,시간,구분,공급량
0,2019-01-01,01,A,2066.196096
1,2019-01-01,02,A,1830.912959
2,2019-01-01,03,A,1757.392366
3,2019-01-01,04,A,1771.582285
4,2019-01-01,05,A,1992.175751


In [48]:
before_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      15120 non-null  object 
 1   시간      15120 non-null  object 
 2   구분      15120 non-null  object 
 3   공급량     15120 non-null  float64
dtypes: float64(1), object(3)
memory usage: 472.6+ KB


In [49]:
present_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      15120 non-null  object 
 1   시간      15120 non-null  object 
 2   구분      15120 non-null  object 
 3   공급량     15120 non-null  float64
dtypes: float64(1), object(3)
memory usage: 472.6+ KB


In [50]:
present_sub['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [51]:
eval_set = pd.DataFrame(['MSE','RMSE','MAE', 'R2','NMAE'], columns=['평가지표'])
for division in present_sub['구분'].unique():
    temp_before_sub = before_sub[before_sub['구분'] == division]
    temp_present_sub = present_sub[present_sub['구분'] == division]

    # 전 제출 값 = 실제값 취급
    # 현제 제출 값 = 예측값
    y = temp_before_sub['공급량']
    pred = temp_present_sub['공급량']
    y_mean = y.mean()

    # MSE
    mse = ( ( y - pred) ** 2 ).sum() / len(pred)
        
    # RMSE
    rmse = mse ** (1/2)
    
    # MAE
    mae = ( abs( y - pred ) ).sum() / len(pred)
    
    # R2
    upper = ( (y - pred) ** 2 ).sum()
    lower = ( (y - y_mean) ** 2 ).sum()
    if lower == 0:
        fraction = 0
    else:
        fraction = upper / lower
    r2 = 1 - fraction
    
    #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
    nmae = ( abs( y - pred ) / y ).sum() / len(pred)
    
    # 평가지표 리스트
    one_eval_set = [mse, rmse, mae, r2, nmae]
    
    # 평가지표 셋에 담기
    eval_set[division] = one_eval_set

# 평가지표 평균 구하기
eval_set['평가지표_평균'] = eval_set.mean(axis=1)

# 평가지표 평균 출력
print(eval_set[['평가지표', '평가지표_평균']])

   평가지표      평가지표_평균
0   MSE  2190.054139
1  RMSE    40.387064
2   MAE    32.224002
3    R2     0.985171
4  NMAE     0.022305


In [52]:
eval_set

,평가지표,A,B,C,D,E,G,H,평가지표_평균
0,MSE,3026.851511,1658.510802,33.138067,1234.066896,3771.750348,5494.986871,111.074476,2190.054139
1,RMSE,55.016829,40.724818,5.756567,35.129288,61.414578,74.128179,10.539188,40.387064
2,MAE,43.452089,32.719819,4.488172,27.922885,49.488741,59.121603,8.374707,32.224002
3,R2,0.986514,0.991228,0.968960,0.983622,0.989088,0.986463,0.990322,0.985171
4,NMAE,0.022617,0.020674,0.025357,0.025959,0.021699,0.019575,0.020256,0.022305


### 평가 지표 함수화

In [53]:
def evaluation_index(before_sub, present_sub):
    eval_set = pd.DataFrame(['MSE','RMSE', 'MAE', 'R2','NMAE'], columns=['평가지표'])
    for division in tqdm(present_sub['구분'].unique()):
        temp_before_sub = before_sub[before_sub['구분'] == division]
        temp_present_sub = present_sub[present_sub['구분'] == division]

        # 전 제출 값 = 실제값 취급
        # 현제 제출 값 = 예측값
        y = temp_before_sub['공급량']
        pred = temp_present_sub['공급량']
        y_mean = y.mean()

        # MSE
        mse = ( ( y - pred) ** 2 ).sum() / len(pred)
            
        # RMSE
        rmse = mse ** (1/2)
        
        # MAE
        mae = ( abs( y - pred ) ).sum() / len(pred)
        
        # R2
        upper = ( (y - pred) ** 2 ).sum()
        lower = ( (y - y_mean) ** 2 ).sum()
        if lower == 0:
            fraction = 0
        else:
            fraction = upper / lower
        r2 = 1 - fraction
        
        #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
        nmae = ( abs( y - pred ) / y ).sum() / len(pred)
        
        # 평가지표 리스트
        one_eval_set = [mse, rmse, mae, r2, nmae]
        
        # 평가지표 셋에 담기
        eval_set[division] = one_eval_set

    # 평가지표 평균 구하기
    eval_set['평가지표_평균'] = eval_set.mean(axis=1)

    # 평가지표 평균 출력
    print(eval_set[['평가지표', '평가지표_평균']])
    
    return eval_set

In [54]:
set = evaluation_index(before_sub, present_sub)

100%|██████████| 7/7 [00:00<00:00, 143.25it/s]

   평가지표      평가지표_평균
0   MSE  2190.054139
1  RMSE    40.387064
2   MAE    32.224002
3    R2     0.985171
4  NMAE     0.022305


In [55]:
set

,평가지표,A,B,C,D,E,G,H,평가지표_평균
0,MSE,3026.851511,1658.510802,33.138067,1234.066896,3771.750348,5494.986871,111.074476,2190.054139
1,RMSE,55.016829,40.724818,5.756567,35.129288,61.414578,74.128179,10.539188,40.387064
2,MAE,43.452089,32.719819,4.488172,27.922885,49.488741,59.121603,8.374707,32.224002
3,R2,0.986514,0.991228,0.968960,0.983622,0.989088,0.986463,0.990322,0.985171
4,NMAE,0.022617,0.020674,0.025357,0.025959,0.021699,0.019575,0.020256,0.022305
